In [9]:
import json 
import os
import datetime
from flask import Flask, jsonify
import hashlib

In [10]:
class ShibaChain:
    def __init__(self):
        self.chain = []
        self.addCoin(proof = 1, prevHash = "0")
        
    def addCoin(self, proof, prevHash):
        coin = {
            "index": len(self.chain),
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "prevHash": prevHash
        }
        self.chain.append(coin)
        return coin
    
    def getLastCoin(self):
        return self.chain[-1]
    
    def proofOfWork(self, prevProof):
        newProof = 1
        solved = False
        while solved is False:
            hashProblem = hashlib.sha256(str(newProof ** 2 - prevProof **2).encode()).hexdigest()
            if hashProblem[:4] == "0000":
                solved = True
            else:
                newProof += 1
        return newProof
    
    def hashCoin(self, coin):
        encodedCoin = json.dumps(coin , sort_keys=True).encode()
        return hashlib.sha256(encodedCoin).hexdigest()
    
    def isChainValid(self, chain):
        previousCoin = chain[0]
        currentIndex = 1
        
        while currentIndex < len(chain):
            currentCoin = chain[currentIndex]
            currentCoinPevHash = currentCoin["prevHash"]
            if currentCoinPevHash != self.hashCoin(previousCoin):
                return False
            currentCoinProof = currentCoin["proof"]
            previousProof = previousCoin["proof"]
            hashEval = hashlib.sha256(str(currentCoinProof ** 2 - previousProof ** 2).encode()).hexdigest()
            if hashEval[:4] != "0000":
                return False
        return True

In [11]:
shiba = ShibaChain()

In [12]:
app = Flask(__name__)

In [13]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousCoin = shiba.getLastCoin()
    previousProof = previousCoin["proof"]
    proof = shiba.proofOfWork(previousProof)
    prevHash = shiba.hashCoin(previousCoin)
    
    newCoin = shiba.addCoin(proof, prevHash)
    response = {
        "success": "you have mined a shiba coin",
        "proof": newCoin["proof"],
        "prevHash": newCoin["prevHash"],
        "timestamp": newCoin["timestamp"],
        "previousCoin": previousCoin,
    }
    return jsonify(response), 200

In [14]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": shiba.chain,
        "length": len(shiba.chain)
    }
    return jsonify(response)

In [15]:
@app.route("/valid", methods = ["GET"])
def valid():
    if shiba.isChainValid(shiba.chain) is False:
        response = {"failed chain": "sorry this is not a proper chain :( "}
    else:
        response = {"chain is Valid": "You have created a proper chain :) "}
    return jsonify(response), 200

In [ ]:
app.run(host = "0.0.0.0", port= 5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Dec/2021 15:30:19] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:20] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:20] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:22] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:22] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:23] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:24] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:25] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:27] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 15:30:28] "GET /chain HTTP/1.1" 200 -
